## ZADATAK

Uvod:

Zadana je mreža cestovnih pravaca između pojedinih gradova (priložena Excel
datoteka). Dva grada su povezana cestom koja osigurava promet u oba smjera.
Između svaka dva grada zadana je udaljenost u [km] i prosječna brzina kretanja u
[km/h].
Kao uvodni dio rada:
• Postojeću mrežu cestovnih pravaca potrebno je proširiti (dopuniti i Excel tablicu)
s 10 novih pravaca po izboru s podacima o udaljenostima i brzini. Pri tome
trebaju biti uključena i najmanje tri nova grada i to sve pod uvjetom da cijeli graf
ostane povezan. Udaljenosti se trebaju uzeti iz nekog stvarnog izvora, a
prosječnu brzinu kretanja treba definirati prema vlastitoj procjeni na temelju
dostupnih podataka o vrsti ceste na pojedinom cestovnom pravcu.
• Na osnovu svih danih i skupljenih podataka preračunati trajanje vožnje u
minutama za svaki pravac.
Zadanu mrežu će biti potrebno predstaviti i obraditi kao graf strukturu gdje gradovi
predstavljaju vrhove grafa, a cestovni pravci bridove grafa. Bridovi su pritom otežani ili
s udaljenošću u kilometrima ili trajanjem vožnje u minutama. Može se koristiti
proizvoljno odabrana implementacija, modul NetworkX, pydot i pygraphviz biblioteke i
sl.

Zadatci:

1. Prikazati listu susjedstva i matricu susjedstva cestovne mreže za oba slučaja
otežanosti pravaca. Napomena: umjesto naziva gradova čvorovi se mogu
označiti skraćenicama (ZG, KT, itd.) radi jednostavnijeg crtanja te je potrebno
uzeti u obzir da se radi o neusmjerenom grafu.
2. Vizualizirati grafove za oba slučaja otežanosti. Može se koristiti NetworkX
modul ili drugi proizvoljan pristup. Primjerice, Python biblioteke pydot i
pygraphviz generiraju slike u PNG formatu.
3. Implementirati funkcionalnost koja će za primjere proizvoljno zadanih polazišta
i odredišta izračunati najkraće putove između njih (dokumentirati najmanje četiri
proizvoljna primjera), i s obzirom na udaljenost u [km] i s obzirom na trajanje
putovanja u [min]. U oba slučaja pored vrijednosti najkraćeg puta dati i redoslijed
svih čvorova na tom putu. Može se koristiti postojeći Python kôd za Dijkstra
algoritam, dati rješenje u bilo kojem drugom jeziku ili uporabom NetworkX
biblioteke.
4. Izgraditi netežinsku inačicu grafa ove cestovne mreže kao primjera povezanosti
vrhova te temeljem toga:
    a. Odrediti dijametar grafa, gustoću grafa i stupnjeve svih čvorova. 
    b. Odrediti mjere centralnosti (one četiri koje su spomenute u materijalima
        s predavanja) svih vrhova grafa

## POČETNI PROGRAMSKI KOD

Sljedeći programski isječaj predstavlja glavnu klasu nad kojom čemo izvršavati operacije poput popunjavanja podataka iz tablice, ispisivanje susjeda u obliku liste i matrice te iscrtavanje samog grafa

In [ ]:
import pandas as pd
import networkx as nx

class Graphs:
    city_codes = {
        "AMSTERDAM": "AM",
        "BARCELONA": "BA",
        "BERLIN": "BE",
        "BRUSSELS": "BR",
        "COLOGNE": "CO",
        "COPENHAGEN": "CO",
        "FRANKFURT": "FR",
        "GENEVA": "GE",
        "HAMBURG": "HA",
        "LISBON": "LI",
        "LONDON": "LO",
        "LUXEMBOURG": "LU",
        "LYON": "LY",
        "MADRID": "MA",
        "MARSEILLE": "MA",
        "MILAN": "MI",
        "MUNICH": "MU",
        "NAPLES": "NA",
        "PARIS": "PA",
        "PRAGUE": "PR",
        "ROME": "RO",
        "VENICE": "VE",
        "VIENNA": "VI",
        "ZURICH": "ZU",
    }

    def __init__(self, excel_file: str):
        self.excel_file = excel_file
        self.table = None
        self.Graph_km = nx.Graph()
        self.Graph_min = nx.Graph()
        self.load_and_prepare()

    def load_and_prepare(self):
        # Učitavanje Excel tablice
        self.table = pd.read_excel(self.excel_file)

        # Preimenovanje stupaca radi jednostavnije upotrebe
        self.table.rename(columns={
            "POLAZIŠTE": "Polaziste",
            "ODREDIŠTE": "Odrediste",
            "UDALJENOST km": "Udaljenost_km",
            "PROSJEČNA BRZ km/h": "Brzina_kmh",
            "TRAJANJE [min]": "Trajanje_min"
        }, inplace=True)

        # Čišćenje naziva gradova i zamjena s kodovima
        self.table["Polaziste"] = self.table["Polaziste"].str.strip().str.upper()
        self.table["Odrediste"] = self.table["Odrediste"].str.strip().str.upper()

        self.table["Polaziste"] = self.table["Polaziste"].apply(lambda x: self.city_codes.get(x, x))
        self.table["Odrediste"] = self.table["Odrediste"].apply(lambda x: self.city_codes.get(x, x))

        # 1. Graf s udaljenošću (km)
        for _, row in self.table.iterrows():
            self.Graph_km.add_edge(row["Polaziste"], row["Odrediste"], weight=row["Udaljenost_km"])

        # 2. Graf s trajanjem (min)
        for _, row in self.table.iterrows():
            self.Graph_min.add_edge(row["Polaziste"], row["Odrediste"], weight=row["Trajanje_min"])


## 1. ZADATAK

Sljedeći programski kod ispisuje susjedne gradove u obliku liste susjedstva i matrice susjedstva.

In [ ]:
import networkx as nx

if __name__ == "__main__":
    graphs = Graphs("MREZA_CESTOVNIH_PRAVACA.xlsx")

    # === LISTA SUSJEDSTVA ===

    print("Lista susjedstva (udaljenost u km):")
    for node in graphs.Graph_km.nodes:
        neighbors = ", ".join([f"{nbr} ({round(d['weight'], 1)} km)" for nbr, d in graphs.Graph_km[node].items()])
        print(f"{node}: {neighbors}")

    print("\nLista susjedstva (trajanje u min):")
    for node in graphs.Graph_min.nodes:
        neighbors = ", ".join([f"{nbr} ({round(d['weight'], 1)} min)" for nbr, d in graphs.Graph_min[node].items()])
        print(f"{node}: {neighbors}")

    # Matrica s udaljenostima
    matrix_km = nx.to_pandas_adjacency(graphs.Graph_km, weight='weight')
    print("\nMatrica susjedstva (udaljenosti u km):")
    print(matrix_km.round(1))

    # Matrica s trajanjem
    matrix_min = nx.to_pandas_adjacency(graphs.Graph_min, weight='weight')
    print("\nMatrica susjedstva (trajanje u minutama):")
    print(matrix_min.round(1))
    



<pre> Lista susjedstva (udaljenost u km):
LO: PA (414 km), BR (328 km), AM (494 km)
PA: LO (414 km), BR (294 km), LU (379 km), GE (546 km), LY (481 km)
BR: LO (328 km), PA (294 km), AM (209 km), LU (233 km), CO (219 km)
AM: LO (494 km), BR (209 km), HA (455 km), BE (649 km)
LU: PA (379 km), BR (233 km), FR (249 km), ZU (438 km)
GE: PA (546 km), ZU (278 km), LY (162 km), MI (412 km)
LY: PA (481 km), GE (162 km), MA (328 km), BA (644 km)
CO: BR (219 km), FR (190 km), BE (743 km), HA (321 km)
HA: AM (455 km), BE (291 km), CO (321 km)
BE: AM (649 km), CO (743 km), HA (291 km), PR (354 km), VI (666 km)
FR: CO (190 km), MU (383 km), ZU (432 km), LU (249 km)
MU: FR (383 km), VI (458 km), ZU (313 km)
ZU: FR (432 km), MU (313 km), GE (278 km), MI (305 km), LU (438 km)
PR: BE (354 km), VI (312 km)
VI: BE (666 km), MU (458 km), PR (312 km), VE (610 km)
MI: ZU (305 km), GE (412 km), RO (681 km), VE (295 km), MA (587 km)
MA: LY (328 km), BA (628 km), LI (638 km), MI (587 km)
BA: LY (644 km), MA (628 km), LI (1266 km)
RO: MI (681 km), NA (233 km)
VE: MI (295 km), VI (610 km)
NA: RO (233 km)
LI: MA (638 km), BA (1266 km)

Lista susjedstva (trajanje u min):
LO: PA (225.8 min), BR (178.9 min), AM (269.5 min)
PA: LO (225.8 min), BR (135.7 min), LU (174.9 min), GE (252.0 min), LY (222.0 min)
BR: LO (178.9 min), PA (135.7 min), AM (104.5 min), LU (116.5 min), CO (109.5 min)
AM: LO (269.5 min), BR (104.5 min), HA (227.5 min), BE (324.5 min)
LU: PA (174.9 min), BR (116.5 min), FR (124.5 min), ZU (238.9 min)
GE: PA (252.0 min), ZU (139.0 min), LY (74.8 min), MI (224.7 min)
LY: PA (222.0 min), GE (74.8 min), MA (151.4 min), BA (322.0 min)
CO: BR (109.5 min), FR (95.0 min), BE (405.3 min), HA (175.1 min)
HA: AM (227.5 min), BE (145.5 min), CO (175.1 min)
BE: AM (324.5 min), CO (405.3 min), HA (145.5 min), PR (193.1 min), VI (363.3 min)
FR: CO (95.0 min), MU (191.5 min), ZU (216.0 min), LU (124.5 min)
MU: FR (191.5 min), VI (249.8 min), ZU (170.7 min)
ZU: FR (216.0 min), MU (170.7 min), GE (139.0 min), MI (166.4 min), LU (238.9 min)
PR: BE (193.1 min), VI (170.2 min)
VI: BE (363.3 min), MU (249.8 min), PR (170.2 min), VE (332.7 min)
MI: ZU (166.4 min), GE (224.7 min), RO (371.5 min), VE (160.9 min), MA (320.2 min)
MA: LY (151.4 min), BA (314.0 min), LI (319.0 min), MI (320.2 min)
BA: LY (322.0 min), MA (314.0 min), LI (690.5 min)
RO: MI (371.5 min), NA (127.1 min)
VE: MI (160.9 min), VI (332.7 min)
NA: RO (127.1 min)
LI: MA (319.0 min), BA (690.5 min)

Matrica susjedstva (udaljenosti u km):
       LO     PA     BR     AM     LU     GE     LY     CO     HA     BE     FR     MU     ZU     PR     VI     MI     MA      BA     RO     VE     NA      LI
LO    0.0  414.0  328.0  494.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
PA  414.0    0.0  294.0    0.0  379.0  546.0  481.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
BR  328.0  294.0    0.0  209.0  233.0    0.0    0.0  219.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
AM  494.0    0.0  209.0    0.0    0.0    0.0    0.0    0.0  455.0  649.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
LU    0.0  379.0  233.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  249.0    0.0  438.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
GE    0.0  546.0    0.0    0.0    0.0    0.0  162.0    0.0    0.0    0.0    0.0    0.0  278.0    0.0    0.0  412.0    0.0     0.0    0.0    0.0    0.0     0.0
LY    0.0  481.0    0.0    0.0    0.0  162.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  328.0   644.0    0.0    0.0    0.0     0.0
CO    0.0    0.0  219.0    0.0    0.0    0.0    0.0    0.0  321.0  743.0  190.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
HA    0.0    0.0    0.0  455.0    0.0    0.0    0.0  321.0    0.0  291.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
BE    0.0    0.0    0.0  649.0    0.0    0.0    0.0  743.0  291.0    0.0    0.0    0.0    0.0  354.0  666.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
FR    0.0    0.0    0.0    0.0  249.0    0.0    0.0  190.0    0.0    0.0    0.0  383.0  432.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
MU    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  383.0    0.0  313.0    0.0  458.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
ZU    0.0    0.0    0.0    0.0  438.0  278.0    0.0    0.0    0.0    0.0  432.0  313.0    0.0    0.0    0.0  305.0    0.0     0.0    0.0    0.0    0.0     0.0
PR    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  354.0    0.0    0.0    0.0    0.0  312.0    0.0    0.0     0.0    0.0    0.0    0.0     0.0
VI    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  666.0    0.0  458.0    0.0  312.0    0.0    0.0    0.0     0.0    0.0  610.0    0.0     0.0
MI    0.0    0.0    0.0    0.0    0.0  412.0    0.0    0.0    0.0    0.0    0.0    0.0  305.0    0.0    0.0    0.0  587.0     0.0  681.0  295.0    0.0     0.0
MA    0.0    0.0    0.0    0.0    0.0    0.0  328.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  587.0    0.0   628.0    0.0    0.0    0.0   638.0
BA    0.0    0.0    0.0    0.0    0.0    0.0  644.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  628.0     0.0    0.0    0.0    0.0  1266.0
RO    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  681.0    0.0     0.0    0.0    0.0  233.0     0.0
VE    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  610.0  295.0    0.0     0.0    0.0    0.0    0.0     0.0
NA    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0  233.0    0.0    0.0     0.0
LI    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  638.0  1266.0    0.0    0.0    0.0     0.0

Matrica susjedstva (trajanje u minutama):
       LO     PA     BR     AM     LU     GE     LY     CO     HA     BE     FR     MU     ZU     PR     VI     MI     MA     BA     RO     VE     NA     LI
LO    0.0  225.8  178.9  269.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
PA  225.8    0.0  135.7    0.0  174.9  252.0  222.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
LO    0.0  225.8  178.9  269.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
PA  225.8    0.0  135.7    0.0  174.9  252.0  222.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
BR  178.9  135.7    0.0  104.5  116.5    0.0    0.0  109.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
PA  225.8    0.0  135.7    0.0  174.9  252.0  222.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
BR  178.9  135.7    0.0  104.5  116.5    0.0    0.0  109.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
AM  269.5    0.0  104.5    0.0    0.0    0.0    0.0    0.0  227.5  324.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
BR  178.9  135.7    0.0  104.5  116.5    0.0    0.0  109.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
AM  269.5    0.0  104.5    0.0    0.0    0.0    0.0    0.0  227.5  324.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
AM  269.5    0.0  104.5    0.0    0.0    0.0    0.0    0.0  227.5  324.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
LU    0.0  174.9  116.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0  124.5    0.0  238.9    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
GE    0.0  252.0    0.0    0.0    0.0    0.0   74.8    0.0    0.0    0.0    0.0    0.0  139.0    0.0    0.0  224.7    0.0    0.0    0.0    0.0    0.0    0.0
LY    0.0  222.0    0.0    0.0    0.0   74.8    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  151.4  322.0    0.0    0.0    0.0    0.0
CO    0.0    0.0  109.5    0.0    0.0    0.0    0.0    0.0  175.1  405.3   95.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
HA    0.0    0.0    0.0  227.5    0.0    0.0    0.0  175.1    0.0  145.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
BE    0.0    0.0    0.0  324.5    0.0    0.0    0.0  405.3  145.5    0.0    0.0    0.0    0.0  193.1  363.3    0.0    0.0    0.0    0.0    0.0    0.0    0.0
FR    0.0    0.0    0.0    0.0  124.5    0.0    0.0   95.0    0.0    0.0    0.0  191.5  216.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
MU    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  191.5    0.0  170.7    0.0  249.8    0.0    0.0    0.0    0.0    0.0    0.0    0.0
ZU    0.0    0.0    0.0    0.0  238.9  139.0    0.0    0.0    0.0    0.0  216.0  170.7    0.0    0.0    0.0  166.4    0.0    0.0    0.0    0.0    0.0    0.0
PR    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  193.1    0.0    0.0    0.0    0.0  170.2    0.0    0.0    0.0    0.0    0.0    0.0    0.0
VI    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  363.3    0.0  249.8    0.0  170.2    0.0    0.0    0.0    0.0    0.0  332.7    0.0    0.0
MI    0.0    0.0    0.0    0.0    0.0  224.7    0.0    0.0    0.0    0.0    0.0    0.0  166.4    0.0    0.0    0.0  320.2    0.0  371.5  160.9    0.0    0.0
MA    0.0    0.0    0.0    0.0    0.0    0.0  151.4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  320.2    0.0  314.0    0.0    0.0    0.0  319.0
BA    0.0    0.0    0.0    0.0    0.0    0.0  322.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  314.0    0.0    0.0    0.0    0.0  690.5
RO    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  371.5    0.0    0.0    0.0    0.0  127.1    0.0
VE    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  332.7  160.9    0.0    0.0    0.0    0.0    0.0    0.0
NA    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  127.1    0.0    0.0    0.0
LI    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  319.0  690.5    0.0    0.0    0.0    0.0 </pre>

## 2. ZADATAK

Sljedeći programski kod prikazuje slike grafova za obje vrste - graf s udaljenostima i graf s trajanjem

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

if __name__ == "__main__":
    graphs = Graphs("MREZA_CESTOVNIH_PRAVACA.xlsx")

    # Prikaz grafa s udaljenostima
    positions = nx.spring_layout(graphs.Graph_km, seed=3, k=100)
    nx.draw(graphs.Graph_km, positions, with_labels=True, node_color='lightblue', node_size=1000, font_size=10)
    edge_labels = nx.get_edge_attributes(graphs.Graph_km, 'weight')
    nx.draw_networkx_edge_labels(graphs.Graph_km, positions, edge_labels={k: f"{v:.1f} km" for k, v in edge_labels.items()})
    plt.title("Cestovna mreža – Udaljenosti (km)")
    plt.show()

    # Prikaz grafa s trajanjem
    positions = nx.spring_layout(graphs.Graph_min, seed=3, k=100)
    nx.draw(graphs.Graph_min, positions, with_labels=True, node_color='lightblue', node_size=1000, font_size=10)
    edge_labels = nx.get_edge_attributes(graphs.Graph_min, 'weight')
    nx.draw_networkx_edge_labels(graphs.Graph_min, positions, edge_labels={k: f"{v:.1f} min" for k, v in edge_labels.items()})
    plt.title("Cestovna mreža – Trajanje (min)")
    plt.show()

    



Graf sa udaljenostima kao otežanosti
![Graf_Km_1](Graf_Km_1.png "Prikaz grafa s udaljenostima (km)")

Graf sa trajanjem vožnje kao otežanosti 
![Graf_Min_2](Graf_Min_2.png "Prikaz grafa s trajanjem (min)")

## 3. ZADATAK

In [ ]:
import networkx as nx

def print_shortest_path(graph, source, target, kriterij_naziv):
    try:
        path = nx.shortest_path(graph, source=source, target=target, weight='weight')
        length = nx.shortest_path_length(graph, source=source, target=target, weight='weight')
        path_str = " -> ".join(path)
        print(f"Najkraći put od {source} do {target} ({kriterij_naziv}):", flush=True)
        print(f"  put: {path_str}", flush=True)
        print(f"  ukupno: {round(length, 1)} {kriterij_naziv}", flush=True)
        print(flush=True)
    except nx.NetworkXNoPath:
        print(f"Nema puta od {source} do {target} ({kriterij_naziv}).\n", flush=True)

parovi = [
    ("ZG", "KT"),
    ("NA", "VT"),
    ("IG", "PŽ"),
    ("DA", "SB"),
]

for polaziste, odrediste in parovi:
    print_shortest_path(graphs.Graph_km, polaziste, odrediste, "km")
    print_shortest_path(graphs.Graph_min, polaziste, odrediste, "min")

Najkraći put od ZG do KT (km):
  put: ZG -> KT
  ukupno: 82.5 km

Najkraći put od ZG do KT (min):
  put: ZG -> KT
  ukupno: 58.0 min

Najkraći put od NA do VT (km):
  put: NA -> VT
  ukupno: 77.2 km

Najkraći put od NA do VT (min):
  put: NA -> VT
  ukupno: 73.0 min

Najkraći put od IG do PŽ (km):
  put: IG -> KT -> NS -> PŽ
  ukupno: 136.8 km

Najkraći put od IG do PŽ (min):
  put: IG -> NS -> PŽ
  ukupno: 111.0 min

Najkraći put od DA do SB (km):
  put: DA -> GR -> KT -> ZG -> SB
  ukupno: 155.5 km

Najkraći put od DA do SB (min):
  put: DA -> GR -> KT -> ZG -> SB
  ukupno: 125.4 min

## 4. ZADATAK

In [ ]:
import networkx as nx
import pandas as pd

# Lista bridova - samo povezanost (bez težina)
edges = [
    ("ZG", "IG"), ("ZG", "KT"), ("ZG", "PŽ"), ("ZG", "ĐK"), ("ZG", "VT"),
    ("ZG", "BJ"), ("ZG", "ŠI"), ("ZG", "SB"), ("ZG", "VŽ"),
    ("IG", "ČZ"), ("IG", "KT"), ("IG", "BJ"), ("IG", "NS"), ("IG", "ĐK"),
    ("KT", "GR"), ("KT", "NS"), ("KT", "VT"), ("KT", "PK"), ("KT", "ČZ"),
    ("PŽ", "NS"), ("PŽ", "DA"), ("PŽ", "PK"), ("PŽ", "VT"), ("PŽ", "ĐK"), ("PŽ", "NA"),
    ("ĐK", "VT"), ("ĐK", "NA"),
    ("VT", "GR"), ("VT", "DA"), ("VT", "BJ"), ("VT", "NA"), ("VT", "NS"),
    ("BJ", "GR"), ("BJ", "ČZ"), ("BJ", "DA"),
    ("GR", "DA"), ("GR", "PK"),
    ("NS", "PK"),
    ("DA", "PK"), ("DA", "NA"),
    ("PK", "NA")
]

# Kreiranje netežinskog grafa
G = nx.Graph()
G.add_edges_from(edges)

# 4a) Dijametar, gustoća, stupnjevi
diameter = nx.diameter(G)
density = nx.density(G)
degrees = dict(G.degree())

# 4b) Centralnosti
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)

# Tablični prikaz rezultata
results = pd.DataFrame({
    "Stupanj": pd.Series(degrees),
    "Centralnost stupnja": pd.Series(degree_centrality),
    "Centralnost međupoloženosti": pd.Series(betweenness_centrality),
    "Centralnost bliskosti": pd.Series(closeness_centrality),
    "Svojstvena centralnost": pd.Series(eigenvector_centrality)
}).sort_values("Stupanj", ascending=False)

# Prikaz osnovnih metrika
print("Osnovne metrike netežinskog grafa:")
print(f"- Dijametar grafa: {diameter}")
print(f"- Gustoća grafa: {round(density, 4)}")

# Prikaz centralnosti
print("\nMjere centralnosti i stupnjevi čvorova:")
display(results)

<pre>Osnovne metrike netežinskog grafa:
- Dijametar grafa: 3
- Gustoća grafa: 0.3417

Mjere centralnosti i stupnjevi čvorova:
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Stupanj</th>
      <th>Centralnost stupnja</th>
      <th>Centralnost međupoloženosti</th>
      <th>Centralnost bliskosti</th>
      <th>Svojstvena centralnost</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>ZG</th>
      <td>9</td>
      <td>0.600000</td>
      <td>0.393175</td>
      <td>0.714286</td>
      <td>0.312488</td>
    </tr>
    <tr>
      <th>VT</th>
      <td>9</td>
      <td>0.600000</td>
      <td>0.116349</td>
      <td>0.714286</td>
      <td>0.400064</td>
    </tr>
    <tr>
      <th>PŽ</th>
      <td>7</td>
      <td>0.466667</td>
      <td>0.073016</td>
      <td>0.625000</td>
      <td>0.324958</td>
    </tr>
    <tr>
      <th>KT</th>
      <td>7</td>
      <td>0.466667</td>
      <td>0.101587</td>
      <td>0.652174</td>
      <td>0.299202</td>
    </tr>
    <tr>
      <th>IG</th>
      <td>6</td>
      <td>0.400000</td>
      <td>0.057460</td>
      <td>0.625000</td>
      <td>0.243027</td>
    </tr>
    <tr>
      <th>BJ</th>
      <td>6</td>
      <td>0.400000</td>
      <td>0.080000</td>
      <td>0.625000</td>
      <td>0.261477</td>
    </tr>
    <tr>
      <th>PK</th>
      <td>6</td>
      <td>0.400000</td>
      <td>0.029048</td>
      <td>0.555556</td>
      <td>0.266932</td>
    </tr>
    <tr>
      <th>DA</th>
      <td>6</td>
      <td>0.400000</td>
      <td>0.023492</td>
      <td>0.555556</td>
      <td>0.282789</td>
    </tr>
    <tr>
      <th>NA</th>
      <td>5</td>
      <td>0.333333</td>
      <td>0.009524</td>
      <td>0.517241</td>
      <td>0.246702</td>
    </tr>
    <tr>
      <th>ĐK</th>
      <td>5</td>
      <td>0.333333</td>
      <td>0.030159</td>
      <td>0.600000</td>
      <td>0.247511</td>
    </tr>
    <tr>
      <th>NS</th>
      <td>5</td>
      <td>0.333333</td>
      <td>0.015714</td>
      <td>0.535714</td>
      <td>0.248636</td>
    </tr>
    <tr>
      <th>GR</th>
      <td>5</td>
      <td>0.333333</td>
      <td>0.011429</td>
      <td>0.535714</td>
      <td>0.244792</td>
    </tr>
    <tr>
      <th>ČZ</th>
      <td>3</td>
      <td>0.200000</td>
      <td>0.001905</td>
      <td>0.468750</td>
      <td>0.130252</td>
    </tr>
    <tr>
      <th>ŠI</th>
      <td>1</td>
      <td>0.066667</td>
      <td>0.000000</td>
      <td>0.428571</td>
      <td>0.050643</td>
    </tr>
    <tr>
      <th>SB</th>
      <td>1</td>
      <td>0.066667</td>
      <td>0.000000</td>
      <td>0.428571</td>
      <td>0.050643</td>
    </tr>
    <tr>
      <th>VŽ</th>
      <td>1</td>
      <td>0.066667</td>
      <td>0.000000</td>
      <td>0.428571</td>
      <td>0.050643</td>
    </tr>
  </tbody>
</table>
</div>
</pre>